### Similarity with LangChain4j
<img src="https://docs.langchain4j.dev/img/logo.svg" alt="LangChain4J" width="200" height="200">

In [10]:
import dev.example.food.*
import dev.langchain4j.model.*
import dev.langchain4j.model.embedding.*
import dev.langchain4j.data.embedding.*
import dev.langchain4j.store.embedding.*
import dev.langchain4j.model.embedding.onnx.allminilml6v2.*
import org.jetbrains.kotlinx.dataframe.api.dataFrameOf



### Embeddings

In [11]:
import dev.langchain4j.model.embedding.onnx.bgesmallzhv15q.BgeSmallZhV15QuantizedEmbeddingModel
val embeddingModel: EmbeddingModel =   BgeSmallZhV15QuantizedEmbeddingModel()


In [29]:
val response = embeddingModel.embed("kingdom")
val embedding:Embedding = response.content()
embedding.dimension()


384

In [8]:
embedding.vector()

[-0.028393552, 0.046382718, -6.550923E-4, -0.0061144317, -0.06512955, -0.06979203, -0.03835533, 0.017542263, 0.047501206, 0.0025471358, 0.05775897, -0.27107093, 0.0070339604, 0.059350982, 0.02203476, 0.02137386, 0.039228108, 0.0579021, -3.0336098E-4, -0.019334145, 0.02721016, 0.014514009, -0.017132826, 0.06159199, 6.6227216E-4, -0.03410037, -0.017672896, -0.060906, -0.05862378, -0.005434127, 0.010982611, -0.031871233, -0.03151921, 0.009996363, -0.04198628, 0.015661, -0.034037765, 0.03993342, 0.01443197, -0.0042922674, -0.032318365, 0.0776131, 0.035826627, -0.053711135, -0.029337745, -0.007351078, 0.050680418, 0.0060712066, -0.058399737, 0.065049514, -0.030843431, 0.04259387, 0.025114441, -0.0014270168, 0.056025207, -0.04645854, 0.02211495, -0.022804009, -0.036363494, 0.08635516, 0.009610418, 0.045615364, 0.024539564, -0.030568631, 0.0067924396, 0.04066138, -0.041816268, 0.037704688, 0.008334117, 0.09546176, -0.045420818, -0.0061681974, 0.044155538, -0.012400166, 0.041659564, 0.00376172

In [9]:
response.tokenUsage() //-> king, ##dom 

TokenUsage { inputTokenCount = 7, outputTokenCount = null, totalTokenCount = 7 }

### Similarity between Vectors
#### Consine Similarity, Euclidean Distance, Manhattan Distance etc.

In [15]:
val embeddingModel = AllMiniLmL6V2EmbeddingModel()
fun similarityBetween(text1:String, text2: String): Double {
    val inputOne: Embedding = embeddingModel.embed(text1).content()
    val inputTwo: Embedding = embeddingModel.embed(text2).content()
    return CosineSimilarity.between(inputOne, inputTwo)
}


In [26]:

similarityBetween("king", "queen")

0.3643842465792434

In [28]:
similarityBetween("the king is in the capital", "the queen is on the throne")

0.501860530034401